In [408]:
import numpy as np
import random
import networkx as nx
from time import time

In [2]:
def calc_M(adj_m):
    '''
    calculates the pagerank matrix from a given adjacency matrix
    
    input: adjacency matrix
    output: pagerank matrix as csr-matrix
    '''
    M = np.array(adj_m, dtype = float).T
    for i in range(len(M)):
        summed = M[:,i].sum()
        if summed == 0: continue
        M[:,i] *= 1/M[:,i].sum()
    return M

In [63]:
def pagerank(beta, M, epsilon, r = 0):
    '''
    calculates pagerank values
    epsilon can also be used as "counter" if integer
    
    
    input:
        r: personalization vector 
            if integer: r is set to default values
        beta: teleportation probability
        M: pagerank matrix (has to be csr)
        epsilon:
            epsilon % 1 = 0: counter for loops
            else: allowed error
    output: pagerank values
    '''
    assert(epsilon >= 0)
    assert(M.shape[0] == M.shape[1])
    if(isinstance(r, int)):
        r = np.array([1/M.shape[0]] *M.shape[0], dtype = float).T
    r_old = r
    while True:
        r_new = (M * beta).dot(r_old)
        D = r_new.sum()
        r_new = r_new + np.array([(1-D)/M.shape[0]] * M.shape[0]).transpose()
        if(epsilon % 1 == 0):
            if(epsilon == 0):
                break
            else:
                epsilon -= 1
        elif(np.linalg.norm(r_new - r_old, ord = 1) < epsilon): 
            break
        r_old = r_new
    return r_new

In [428]:
def generate(size, add_edges = 0):
    '''
    generates the simplest strongly connected aperiodic adjacency matrix I could think of
    example: size = 3, add_edges = 0
        0 1 0
        1 0 1
        1 1 0
    example[2][0] == 1 to ensure aperiodicity
    
    add edges just adds random, non-loop edges (-> off diagonal) 
    '''
    adj_m = np.zeros((size,size))
    adj_m[0][1] = 1
    adj_m[-1][-2] = 1
    adj_m[-1][0] = 1
    for i in range(1, len(adj_m) - 1):
        adj_m[i][i+1] = 1
        adj_m[i][i-1] = 1
    for i in range(add_edges):
        rand_1 = random.randint(0,size - 1)
        rand_2 = random.randint(0, size - 1)
        while(rand_1 == i or rand_2 == i): #no edge to itself
            rand_1 = random.randint(0, size - 1)
            rand_2 = random.randint(0, size - 1)
        adj_m[rand_1][rand_2] = 1
    return adj_m

In [65]:
def power_it(google_matrix, epsilon):
    #breakpoint()
    r_old = np.array([1/google_matrix.shape[0]] * google_matrix.shape[0], dtype = float)
    while True:
        r_new = google_matrix.dot(r_old)
        #r_new = np.dot(google_matrix, r)
        if(epsilon % 1 == 0):
            if(epsilon == 0):
                break
            else:
                epsilon -= 1
        elif(abs(r_new - r_old).sum() < epsilon): 
            break
        r_old = r_new
        
    return r_new

In [384]:
print("example from lecture:\n")
adj_m = np.matrix([[0,1,0,0,0],[1,0,1,0,0],[0,0,0,1,1],[0,0,0,0,0],[0,1,0,0,0]], dtype = float)
pr_matrix = calc_M(adj_m)
for i in range(10):
    print("iterations: ", i + 1 ," \n ", pagerank(r = 0, beta = 0.85, M = pr_matrix, epsilon = i))

example from lecture:

iterations:  1  
  [0.149 0.404 0.149 0.149 0.149]
iterations:  2  
  [0.22703  0.30863  0.22703  0.118655 0.118655]
iterations:  3  
  [0.1813391 0.3440036 0.1813391 0.1466591 0.1466591]
iterations:  4  
  [0.20113358 0.33373052 0.20113358 0.13200116 0.13200116]
iterations:  5  
  [0.19427567 0.33560473 0.19427567 0.13792197 0.13792197]
iterations:  6  
  [0.19607874 0.33581473 0.19607874 0.13601389 0.13601389]
iterations:  7  
  [0.19584362 0.3354011  0.19584362 0.13645583 0.13645583]
iterations:  8  
  [0.19574296 0.33565202 0.19574296 0.13643103 0.13643103]
iterations:  9  
  [0.19584538 0.33554117 0.19584538 0.13638403 0.13638403]
iterations:  10  
  [0.19579028 0.33558029 0.19579028 0.13641957 0.13641957]


In [387]:
print("example from lecture with varying beta \n")
for i in np.arange(0,1.1,0.1):
    print("beta = ", i ," \n ",pagerank(r = 0, beta = i, M = pr_matrix, epsilon = 0.01))

example from lecture with varying beta 

beta =  0.0  
  [0.2 0.2 0.2 0.2 0.2]
beta =  0.1  
  [0.19508 0.22268 0.19508 0.19358 0.19358]
beta =  0.2  
  [0.1917248 0.2431808 0.1917248 0.1866848 0.1866848]
beta =  0.30000000000000004  
  [0.1897112 0.2616752 0.1897112 0.1794512 0.1794512]
beta =  0.4  
  [0.18948915 0.27806259 0.18948915 0.17147955 0.17147955]
beta =  0.5  
  [0.19007  0.29297  0.19007  0.163445 0.163445]
beta =  0.6000000000000001  
  [0.19052217 0.30669953 0.19052217 0.15612806 0.15612806]
beta =  0.7000000000000001  
  [0.19247016 0.31912707 0.19247016 0.14796631 0.14796631]
beta =  0.8  
  [0.19474787 0.33048035 0.19474787 0.14001195 0.14001195]
beta =  0.9  
  [0.19718501 0.34033137 0.19718501 0.13264931 0.13264931]
beta =  1.0  
  [0.20011136 0.34941056 0.20011136 0.12518336 0.12518336]


In [393]:
r = np.array([0.9,0.02,0.02,0.02,0.02], dtype = float)
print("lecture example with new personalization vector")
print("iterations: ", 100)
print("personalization vector: ", r, "\n")
for i in np.arange(0,1.1,0.1):
    #beta = 0 => outgoing edges always chosen randomly
    print("beta: ",i, "\n ",pagerank(r = r, beta = i, M = pr_matrix, epsilon = 100)) # epsilon = 5 

lecture example with new personalization vector
iterations:  100
personalization vector:  [0.9  0.02 0.02 0.02 0.02] 

beta:  0.0 
  [0.2 0.2 0.2 0.2 0.2]
beta:  0.1 
  [0.19500924 0.22273567 0.19500924 0.19362292 0.19362292]
beta:  0.2 
  [0.19178082 0.24315068 0.19178082 0.18664384 0.18664384]
beta:  0.30000000000000004 
  [0.1899841  0.26152623 0.1899841  0.17925278 0.17925278]
beta:  0.4 
  [0.18934911 0.27810651 0.18934911 0.17159763 0.17159763]
beta:  0.5 
  [0.18965517 0.29310345 0.18965517 0.1637931  0.1637931 ]
beta:  0.6000000000000001 
  [0.19072165 0.30670103 0.19072165 0.15592784 0.15592784]
beta:  0.7000000000000001 
  [0.19240048 0.31905911 0.19240048 0.14806996 0.14806996]
beta:  0.8 
  [0.19457014 0.33031674 0.19457014 0.14027149 0.14027149]
beta:  0.9 
  [0.19713071 0.34059511 0.19713071 0.13257173 0.13257173]
beta:  1.0 
  [0.2   0.35  0.2   0.125 0.125]


In [395]:
lecture_graph = nx.DiGraph(adj_m)
pr_val = nx.pagerank(lecture_graph)
print("lecture example with networks:", "\n ",pr_val)

lecture example with networks: 
  {0: 0.1958071483019602, 1: 0.3355711792554186, 2: 0.1958071483019602, 3: 0.13640726207033055, 4: 0.13640726207033055}


In [398]:
print("lecture example with network x and verifying alpha")
for i in np.arange(0,1.1,0.1):
    print("alpha: ", i, "\n ",nx.pagerank(G = lecture_graph, alpha = i))
#same results as our implementation                                   

lecture example with network x and verifying alpha
alpha:  0.0 
  {0: 0.2, 1: 0.2, 2: 0.2, 3: 0.2, 4: 0.2}
alpha:  0.1 
  {0: 0.19500923744000004, 1: 0.22273567424000004, 2: 0.19500923744000004, 3: 0.19362292544000004, 4: 0.19362292544000004}
alpha:  0.2 
  {0: 0.19178083205120006, 1: 0.2431506993152001, 2: 0.19178083205120006, 3: 0.18664381829120005, 4: 0.18664381829120005}
alpha:  0.30000000000000004 
  {0: 0.18998411871987197, 1: 0.261526180051712, 2: 0.18998411871987197, 3: 0.179252791254272, 4: 0.179252791254272}
alpha:  0.4 
  {0: 0.18934902095347714, 1: 0.27810661596004355, 2: 0.18934902095347714, 3: 0.17159767106650112, 4: 0.17159767106650112}
alpha:  0.5 
  {0: 0.1896553907, 1: 0.29310328220000004, 2: 0.1896553907, 3: 0.1637929682, 4: 0.1637929682}
alpha:  0.6000000000000001 
  {0: 0.1907217457756634, 1: 0.30670100504713005, 2: 0.1907217457756634, 3: 0.1559277517007716, 4: 0.1559277517007716}
alpha:  0.7000000000000001 
  {0: 0.19240035301429034, 1: 0.3190590930876257, 2: 0.19

In [431]:
size = 10000
additional_edges = 5000
iterations = 30
tol = 0.01

In [434]:
gen_start = time()
test_m = generate(size = size, add_edges = additional_edges) 
gen_end = time()

gm_start = time()
google_m = calc_M(test_m)
gm_end = time()

pii_start = time()
powerit = power_it(google_m, epsilon = 30)
pii_end = time()

pri_start = time()
pager = pagerank(beta = 0.85, M = google_m, epsilon = 30)
pri_end = time()

pit_start = time()
powerit2 = power_it(google_m, epsilon = 0.01)
pit_end = time()

prt_start = time()
pager2 = power_it(google_m, epsilon = 0.01)
prt_end = time()

nx_start = time()
nxp = nx.pagerank(nx.DiGraph(test_m))
nx_end = time()



print("n =", size, "iterations =", iterations, "epsilon =" tol, "added edges =", additional_edges)
print("generation: ", gen_end - gen_start)
print("google_matrix calculation: ", gm_end - gm_start)
print("power-iteration with max_iter: ", pii_end - pii_start)
print("pagerank with max_iter: ", pri_end - pri_start)
print("power-iteration with epsilon: ", pit_end - pit_start)
print("pagerank with epsilon: ", prt_end - prt_start)



n =  10000 iterations = 30 epsilon =  0.01 added edges =   500
generation:  0.09199714660644531
google_matrix calculation:  0.7340009212493896
power-iteration with max_iter:  1.1334912776947021
pagerank with max_iter:  10.296735525131226
power-iteration with epsilon:  0.8160014152526855
pagerank with epsilon:  0.7739994525909424


In [402]:
## Never used

def generate_adj_m(size, deadends = False, loops = True, max_edges_vertex = 1):
    '''
    generates adjacency matrx
    
    input:
        size: number of vertices
        deadends: ensures vertices with no outgoing edges if true
        loops: if false ensures no vertex has an edge to itself
        max_edges_vertex: amount of outgoing edges per vertex
    
    output: adjacency matrix
    '''
    assert(size > 1)
    adj_m = np.zeros((size,size))
    for row in adj_m:
        for amount_edges in range(random.randint(1,max_edges_vertex)):
            row[random.randint(0, size - 1)] = 1
    if deadends:
        for i in range(0, random.randint(1, size // 2)):
            adj_m[random.randint(0, size - 1)] = np.zeros(size)
    if(not loops):
        for i in range(size):
            while(adj_m[i][i]):
                adj_m[i][i] = 0
                adj_m[i][random.randint(0, size - 1)] = 1
        
    return adj_m